In [7]:
import numpy as np
import pandas as pd
import xarray as xr
import geopandas as gpd
import vwf.data as vwf_data
from calendar import monthrange
def daysDuringMonth(yy, m):
    """
    Attach number of days in month to each year in yy for month m.
    """
    result = []    
    [result.append(monthrange(y, m)[1]) for y in yy]        
    return result

# FRANCE

In [3]:
fr_md = gpd.read_file("input/country-data/fr/fr_turb_info.csv")
fr_md = fr_md.loc[fr_md['statut_parc'] == 'Autorisé'].reset_index(drop=True)
fr_md = fr_md.loc[
    :, [
    "id_aerogenerateur", 
    "puissance_mw", 
    "diametre_rotor",
    "hauteur_mat_nacelle",

    "constructeur",
    "x_aerogenerateur",
    "y_aerogenerateur",
    "epsg"
    ]]
fr_md.columns = [
    "ID",
    "capacity",
    "diameter",
    "height",
    "manufacturer",
    "x",
    "y",
    "epsg"
    ]

points_gdf = gpd.GeoDataFrame(
    fr_md[["ID","capacity","diameter","height","manufacturer"]],
    geometry=gpd.points_from_xy(fr_md.x, fr_md.y, crs=fr_md.epsg.iloc[0])
    ).to_crs(epsg=4326)

points_gdf['capacity'] = points_gdf['capacity'].astype(float) * 1e3  # MW to kW
points_gdf['lon'] = points_gdf.geometry.x  
points_gdf['lat'] = points_gdf.geometry.y
points_gdf = points_gdf.drop(columns='geometry')

# Convert to desired crs and save directly to a shapefile

turb_info = vwf_data.add_models(points_gdf)

Total observed turbines/farms before conditions:  10276


In [ ]:
# ALTERNATIVE FR GENERATION DATA
# https://ec.europa.eu/eurostat/databrowser/view/nrg_cb_pem__custom_19402431/default/table
year_star = 2015 # start year of training period
year_end = 2018
ns_data = pd.read_csv("input/country-data/northsea_country_generation.csv")
ns_data = ns_data.loc[
    :, [
    "Standard international energy product classification (SIEC)", 
    "TIME_PERIOD", 
    "OBS_VALUE",
    "geo",
    ]]
ns_data.columns = [
    "carrier",
    "date",
    "output",
    "country",
    ]

country = 'FR'  # Example country code
ns_data = ns_data.loc[(ns_data['country']==country) & (ns_data['carrier']=='Wind')].reset_index(drop=True)
ns_data['date'] = pd.to_datetime(ns_data['date'])
# convert from gigawatt hours to kilowatt hours
ns_data['output'] = pd.to_numeric(ns_data['output'])
ns_data['output'] = ns_data['output'] * 1e6
ns_data['year'] = ns_data['date'].dt.year.astype(int)
ns_data['month'] = ns_data['date'].dt.month.astype(int)
ns_data = ns_data.drop(columns=['date'])

ns_data = ns_data.fillna(0).groupby(['year','month'])['output'].sum().reset_index()

turb_info["ratio"] = turb_info['capacity'] / turb_info['capacity'].sum()

ns_data = ns_data.merge(turb_info[['ID', 'ratio']], how="cross")
ns_data["output"] = ns_data["output"] * ns_data["ratio"]
ns_data = ns_data.dropna(subset=['ID', 'year', 'month'])
ns_data = ns_data.loc[(ns_data["year"] >= year_star) & (ns_data["year"] <= year_end)].reset_index(drop=True)   
obs_gen = ns_data.pivot(index=['ID','year'], columns='month', values='output').reset_index()
# obs_gen = ns_data.pivot(index=['type','year'], columns='month', values='output').reset_index()
obs_gen.columns = [f'obs_{i}' if i not in ['ID', 'year'] else f'{i}' for i in obs_gen.columns]
obs_gen = obs_gen.merge(turb_info[['ID', 'capacity']], how='left', on=['ID'])
obs_gen = obs_gen.dropna().reset_index(drop=True)



for i in range(1,13):
    obs_gen['obs_'+str(i)] = obs_gen['obs_'+str(i)]/(((daysDuringMonth(obs_gen.year, i))*obs_gen['capacity'])*24)

obs_gen = obs_gen.drop(['capacity'], axis=1)
obs_gen = obs_gen.sort_values(by=['ID','year']).reset_index(drop=True)
obs_gen

,ID,year,obs_1,obs_2,obs_3,obs_4,obs_5,obs_6,obs_7,obs_8,obs_9,obs_10,obs_11,obs_12
0,0000000044_E1,2015,0.112989,0.103939,0.099125,0.073938,0.078874,0.064387,0.069476,0.055871,0.085045,0.063245,0.130545,0.131214
1,0000000044_E1,2016,0.140924,0.152475,0.121089,0.085957,0.076693,0.054944,0.055716,0.061687,0.050490,0.072020,0.123569,0.072072
2,0000000044_E1,2017,0.099114,0.139221,0.130560,0.073974,0.072721,0.074893,0.084057,0.060763,0.083579,0.102415,0.116556,0.160125
3,0000000044_E1,2018,0.196306,0.149803,0.151215,0.109189,0.077720,0.067985,0.052396,0.066873,0.079519,0.119011,0.137384,0.176818
4,0000000044_E2,2015,0.112989,0.103939,0.099125,0.073938,0.078874,0.064387,0.069476,0.055871,0.085045,0.063245,0.130545,0.131214
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
40511,0100283090_E7,2018,0.196306,0.149803,0.151215,0.109189,0.077720,0.067985,0.052396,0.066873,0.079519,0.119011,0.137384,0.176818
40512,0100283090_E8,2015,0.112989,0.103939,0.099125,0.073938,0.078874,0.064387,0.069476,0.055871,0.085045,0.063245,0.130545,0.131214
40513,0100283090_E8,2016,0.140924,0.152475,0.121089,0.085957,0.076693,0.054944,0.055716,0.061687,0.050490,0.072020,0.123569,0.072072
40514,0100283090_E8,2017,0.099114,0.139221,0.130560,0.073974,0.072721,0.074893,0.084057,0.060763,0.083579,0.102415,0.116556,0.160125


In [ ]:
turb_info.lat.min(), turb_info.lat.max(), turb_info.lon.min(), turb_info.lon.max()

(42.50154105996311, 50.930297624283845, -4.757239614418522, 8.7060513884254)

In [ ]:
year_star = 2015 # start year of training period
year_end = 2018 

fr_data = pd.read_csv("input/country-data/FR/observations/Wind_power_generation_in_France.csv")
fr_data.columns = ["date", "filter", "output", "nature"]

fr_data = fr_data.loc[fr_data['filter'].str.contains('evolution')].reset_index(drop=True)
# fr_data["type"] = fr_data["filter"].str.split(" ").str[0].str.lower()
fr_data["output"] = fr_data["output"].replace(',', '.', regex=True)
fr_data = fr_data.drop(columns=["nature","filter"])
fr_data['date'] = pd.to_datetime(fr_data['date'])
fr_data['output'] = pd.to_numeric(fr_data['output'])
# convert from terawatts to kilowatts
fr_data['output'] = fr_data['output'] * 1e9
fr_data['year'] = fr_data['date'].dt.year.astype(int)
fr_data['month'] = fr_data['date'].dt.month.astype(int)
fr_data = fr_data.drop(columns=['date'])

fr_data = fr_data.fillna(0).groupby(['year','month'])['output'].sum().reset_index()
# fr_data['type'] = 'onshore'

turb_info["ratio"] = turb_info['capacity'] / turb_info['capacity'].sum()

fr_data = fr_data.merge(turb_info[['ID', 'ratio']], how="cross")
fr_data["output"] = fr_data["output"] * fr_data["ratio"]
fr_data = fr_data.dropna(subset=['ID', 'year', 'month'])
fr_data = fr_data.loc[(fr_data["year"] >= year_star) & (fr_data["year"] <= year_end)].reset_index(drop=True)   
obs_gen = fr_data.pivot(index=['ID','year'], columns='month', values='output').reset_index()
# obs_gen = fr_data.pivot(index=['type','year'], columns='month', values='output').reset_index()

obs_gen.columns = [f'obs_{i}' if i not in ['ID', 'year'] else f'{i}' for i in obs_gen.columns]
obs_gen = obs_gen.merge(turb_info[['ID', 'capacity']], how='left', on=['ID'])
obs_gen = obs_gen.dropna().reset_index(drop=True)

for i in range(1,13):
    obs_gen['obs_'+str(i)] = obs_gen['obs_'+str(i)]/(((daysDuringMonth(obs_gen.year, i))*obs_gen['capacity'])*24)


obs_gen = obs_gen.drop(['capacity'], axis=1)
obs_gen = obs_gen.sort_values(by=['ID','year']).reset_index(drop=True)
obs_gen

,ID,year,obs_1,obs_2,obs_3,obs_4,obs_5,obs_6,obs_7,obs_8,obs_9,obs_10,obs_11,obs_12
0,0000000044_E1,2015,0.119402,0.109819,0.104127,0.077930,0.082066,0.067322,0.072649,0.058472,0.089848,0.067093,0.136548,0.133591
1,0000000044_E1,2016,0.148005,0.160959,0.127405,0.089613,0.080062,0.057473,0.052848,0.061672,0.053158,0.073204,0.124678,0.079410
2,0000000044_E1,2017,0.106120,0.145694,0.137468,0.078337,0.077585,0.078801,0.088521,0.063991,0.087904,0.107737,0.124232,0.174759
3,0000000044_E1,2018,0.207637,0.159707,0.162313,0.115942,0.083022,0.073531,0.057142,0.072849,0.086501,0.129369,0.150310,0.189606
4,0000000044_E2,2015,0.119402,0.109819,0.104127,0.077930,0.082066,0.067322,0.072649,0.058472,0.089848,0.067093,0.136548,0.133591
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
40511,0100283090_E7,2018,0.207637,0.159707,0.162313,0.115942,0.083022,0.073531,0.057142,0.072849,0.086501,0.129369,0.150310,0.189606
40512,0100283090_E8,2015,0.119402,0.109819,0.104127,0.077930,0.082066,0.067322,0.072649,0.058472,0.089848,0.067093,0.136548,0.133591
40513,0100283090_E8,2016,0.148005,0.160959,0.127405,0.089613,0.080062,0.057473,0.052848,0.061672,0.053158,0.073204,0.124678,0.079410
40514,0100283090_E8,2017,0.106120,0.145694,0.137468,0.078337,0.077585,0.078801,0.088521,0.063991,0.087904,0.107737,0.124232,0.174759


# NETHERLANDS

In [8]:
# https://nationaalgeoregister.nl/geonetwork/srv/dut/catalog.search#/metadata/90f5eab6-9cea-4869-a031-2a228fb82fea
nl_md = gpd.read_file("input/country-data/NL/nl_md.json").to_crs(epsg=4326)
nl_md['lon'] = nl_md.geometry.x  
nl_md['lat'] = nl_md.geometry.y
nl_md = nl_md.drop(columns=['geometry','x','y','prov_naam','gem_naam','naam'])
nl_md["ondergrond"] = nl_md["ondergrond"].replace({"land": "onshore", "zee": "offshore"})
nl_md["land"] = nl_md["land"].replace({"België": "BE", "Duitsland": "DE", "Nederland": "NL"})
nl_md.columns = [
    "ID",
    "diameter",
    "height",
    "capacity",
    "country",
    "manufacturer",
    "type",
    "lon",
    "lat"
]
nl_md
# nl_md = nl_md.loc[nl_md['country']=='NL'].reset_index(drop=True).drop(columns=['country'])
# nl_md = nl_md[["ID","capacity","diameter","height","manufacturer","lon","lat","type"]]
# nl_md['manufacturer'] = nl_md['manufacturer'].str.split(' ').str[0].str.strip('123-.,').astype(str)
# turb_info = vwf_data.add_models(nl_md)
# turb_info
# nl_md['manufacturer'].str.strip().unique()

DataSourceError: input/country-data/NL/nl_md.json: No such file or directory

In [ ]:
# https://ec.europa.eu/eurostat/databrowser/view/nrg_cb_pem__custom_19402431/default/table
year_star = 2015 # start year of training period
year_end = 2018
ns_data = pd.read_csv("input/country-data/northsea_country_generation.csv")
ns_data = ns_data.loc[
    :, [
    "Standard international energy product classification (SIEC)", 
    "TIME_PERIOD", 
    "OBS_VALUE",
    "geo",
    ]]
ns_data.columns = [
    "carrier",
    "date",
    "output",
    "country",
    ]

country = 'NL'  # Example country code
ns_data = ns_data.loc[(ns_data['country']==country) & (ns_data['carrier']=='Wind')].reset_index(drop=True)
ns_data['date'] = pd.to_datetime(ns_data['date'])
# convert from gigawatt hours to kilowatt hours
ns_data['output'] = pd.to_numeric(ns_data['output'])
ns_data['output'] = ns_data['output'] * 1e6
ns_data['year'] = ns_data['date'].dt.year.astype(int)
ns_data['month'] = ns_data['date'].dt.month.astype(int)
ns_data = ns_data.drop(columns=['date'])

ns_data = ns_data.fillna(0).groupby(['year','month'])['output'].sum().reset_index()

turb_info["ratio"] = turb_info['capacity'] / turb_info['capacity'].sum()

ns_data = ns_data.merge(turb_info[['ID', 'ratio']], how="cross")
ns_data["output"] = ns_data["output"] * ns_data["ratio"]
ns_data = ns_data.dropna(subset=['ID', 'year', 'month'])
ns_data = ns_data.loc[(ns_data["year"] >= year_star) & (ns_data["year"] <= year_end)].reset_index(drop=True)   
obs_gen = ns_data.pivot(index=['ID','year'], columns='month', values='output').reset_index()
# obs_gen = ns_data.pivot(index=['type','year'], columns='month', values='output').reset_index()
obs_gen.columns = [f'obs_{i}' if i not in ['ID', 'year'] else f'{i}' for i in obs_gen.columns]
obs_gen = obs_gen.merge(turb_info[['ID', 'capacity']], how='left', on=['ID'])
obs_gen = obs_gen.dropna().reset_index(drop=True)



for i in range(1,13):
    obs_gen['obs_'+str(i)] = obs_gen['obs_'+str(i)]/(((daysDuringMonth(obs_gen.year, i))*obs_gen['capacity'])*24)

obs_gen = obs_gen.drop(['capacity'], axis=1)
obs_gen = obs_gen.sort_values(by=['ID','year']).reset_index(drop=True)
obs_gen

NameError: name 'pd' is not defined

In [12]:
year_star = 2015 # start year of training period
year_end = 2018
ns_data = pd.read_csv("../input/country-data/northsea_country_generation.csv")
ns_data = ns_data.loc[
    :, [
    "Standard international energy product classification (SIEC)", 
    "TIME_PERIOD", 
    "OBS_VALUE",
    "geo",
    ]]
ns_data.columns = [
    "carrier",
    "date",
    "output",
    "country",
    ]
ns_data.country.unique()

array(['BE', 'FR', 'NL', 'NO'], dtype=object)

In [5]:
turb_info.lat.min(), turb_info.lat.max(), turb_info.lon.min(), turb_info.lon.max()

(50.84785605236023, 54.073136499178204, 2.774671661010408, 7.1778683054236145)

In [91]:
# https://data.open-power-system-data.org/time_series.com
data = pd.read_csv("input/country-data/time_series_60min_singleindex_filtered.csv")
data

,utc_timestamp,cet_cest_timestamp,BE_wind_generation_actual,BE_wind_offshore_generation_actual,BE_wind_onshore_generation_actual,FR_wind_onshore_generation_actual,NL_wind_generation_actual,NL_wind_offshore_generation_actual,NL_wind_onshore_generation_actual,NO_wind_onshore_generation_actual,NO_1_wind_onshore_generation_actual,NO_2_wind_onshore_generation_actual,NO_3_wind_onshore_generation_actual,NO_4_wind_onshore_generation_actual,NO_5_wind_onshore_generation_actual
0,2014-12-31T23:00:00Z,2015-01-01T00:00:00+0100,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2015-01-01T00:00:00Z,2015-01-01T01:00:00+0100,NaN,NaN,NaN,NaN,1451.0,145.0,1306.0,NaN,NaN,NaN,NaN,NaN,NaN
2,2015-01-01T01:00:00Z,2015-01-01T02:00:00+0100,734.81,518.66,216.15,1464.0,1447.0,145.0,1302.0,479.40,NaN,158.60,233.53,68.67,18.6
3,2015-01-01T02:00:00Z,2015-01-01T03:00:00+0100,766.64,529.46,237.18,1543.0,1479.0,148.0,1331.0,422.74,NaN,149.20,200.27,54.67,18.6
4,2015-01-01T03:00:00Z,2015-01-01T04:00:00+0100,733.13,406.94,326.19,1579.0,1340.0,134.0,1206.0,408.35,NaN,143.50,192.22,54.03,18.6
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
50396,2020-09-30T19:00:00Z,2020-09-30T21:00:00+0200,1889.72,1497.40,392.32,3632.0,870.0,589.0,281.0,1533.20,94.20,964.93,376.02,98.05,NaN
50397,2020-09-30T20:00:00Z,2020-09-30T22:00:00+0200,2154.67,1688.06,466.61,3965.0,978.0,650.0,328.0,1645.93,86.20,1034.32,425.09,100.32,NaN
50398,2020-09-30T21:00:00Z,2020-09-30T23:00:00+0200,2187.48,1715.76,471.72,4201.0,988.0,705.0,284.0,1635.35,80.69,1008.48,436.27,109.91,NaN
50399,2020-09-30T22:00:00Z,2020-10-01T00:00:00+0200,2225.62,1739.17,486.45,4428.0,912.0,662.0,250.0,1698.59,84.76,1003.72,505.29,104.82,NaN


# DATA API

In [13]:
"""
Norway wind metadata from NVE (operational plants) -> pseudo-turbines.

Outputs CSV with columns:
["ID","capacity","diameter","height","manufacturer","lon","lat","type"]

Key idea:
- NVE returns plant-level records, often with "Turbiner" blocks.
- Each block typically has AntallTurbiner (count) and TurbinStorrelse_kW (per-turbine size).
- We expand each block into N pseudo-turbine rows at the plant centroid (same lon/lat).
"""

from __future__ import annotations

import re
import sys
import math
import time
import requests
import pandas as pd

SCHEMA = ["ID", "capacity", "diameter", "height", "manufacturer", "lon", "lat", "type"]

NVE_ENDPOINT = "https://api.nve.no/web/WindPowerplant/GetWindPowerPlantsInOperation"


# -------------------------
# Helpers
# -------------------------
def _first(d: dict, keys: list[str], default=None):
    for k in keys:
        if k in d and d[k] not in (None, "", "null"):
            return d[k]
    return default


def _to_float(x):
    if x is None:
        return None
    if isinstance(x, (int, float)):
        return float(x)
    s = str(x).strip()
    if not s:
        return None
    m = re.search(r"[-+]?\d*\.?\d+", s.replace(",", "."))
    return float(m.group(0)) if m else None


def _to_int(x):
    f = _to_float(x)
    if f is None or math.isnan(f):
        return None
    return int(round(f))


def fetch_nve_in_operation(retries: int = 3, timeout: int = 60) -> list[dict]:
    last_err = None
    for i in range(retries):
        try:
            r = requests.get(NVE_ENDPOINT, timeout=timeout)
            r.raise_for_status()
            data = r.json()
            if not isinstance(data, list):
                raise ValueError(f"Unexpected JSON root type: {type(data)}")
            return data
        except Exception as e:
            last_err = e
            time.sleep(2 + i * 2)
    raise last_err


def _get_lon_lat(plant: dict) -> tuple[float | None, float | None]:
    # NVE field names can vary; try a few common ones
    lon = _first(plant, ["Longitude", "lon", "Long", "X", "x", "x_coord", "x_coordinaat"])
    lat = _first(plant, ["Latitude", "lat", "Lat", "Y", "y", "y_coord", "y_coordinaat"])
    return _to_float(lon), _to_float(lat)


def _get_plant_id(plant: dict) -> str:
    pid = _first(plant, ["VindkraftAnleggId", "anleggId", "id", "ID"])
    return str(pid) if pid is not None else "unknown"


def _guess_block_diameter(block: dict) -> float | None:
    # try a few plausible keys
    return _to_float(_first(block, ["RotorDiameter", "rotor_diameter", "rotordiameter", "Rotor_diameter"]))


def _guess_block_hub_height(block: dict) -> float | None:
    return _to_float(_first(block, ["NavHoyde", "HubHeight", "hub_height", "ashoogte", "Hub_height"]))


def _guess_block_manufacturer(block: dict) -> str | None:
    m = _first(block, ["Produsent", "Manufacturer", "manufacturer", "Maker", "maker", "brand"])
    if m is None:
        return None
    s = str(m).strip()
    return s if s else None


def _guess_block_count(block: dict) -> int | None:
    return _to_int(_first(block, ["AntallTurbiner", "count", "Count", "number", "Number"]))


def _guess_block_size_kw(block: dict) -> float | None:
    # per-turbine size in kW is often here
    return _to_float(_first(block, ["TurbinStorrelse_kW", "Size_kW", "size_kw", "kW"]))


def _guess_plant_installed_mw(plant: dict) -> float | None:
    return _to_float(_first(plant, ["InstallertEffekt_MW", "installed_mw", "InstalledMW", "capacity_mw"]))


# -------------------------
# Main transformation
# -------------------------
def nve_to_pseudoturbines(
    data: list[dict],
    *,
    expand: bool = True,
    fallback_single_if_missing_count: bool = True,
) -> pd.DataFrame:
    """
    Convert NVE JSON to pseudo-turbine dataframe.

    - If expand=True:
        For each plant turbine block with count N, create N rows.
        Each row has per-turbine capacity (MW) from (TurbinStorrelse_kW / 1000).
        Diameter/height/manufacturer copied from block if present.
        Coordinates are plant centroid.

    - If a plant has no Turbiner blocks:
        Create one row at plant level (capacity from InstallertEffekt_MW).
    """
    rows: list[dict] = []

    for plant in data:
        pid = _get_plant_id(plant)
        lon, lat = _get_lon_lat(plant)

        # Some records may not have coordinates; keep them (but they may be dropped later)
        # You can choose to skip, but keeping is safer for debugging.
        plant_installed_mw = _guess_plant_installed_mw(plant)

        blocks = plant.get("Turbiner") or []

        if isinstance(blocks, list) and blocks and expand:
            for b_i, block in enumerate(blocks):
                n = _guess_block_count(block)
                size_kw = _guess_block_size_kw(block)
                diameter = _guess_block_diameter(block)
                hub_height = _guess_block_hub_height(block)
                manufacturer = _guess_block_manufacturer(block)

                # per-turbine capacity in MW if size_kw present
                cap_per_turb_mw = (size_kw / 1000.0) if size_kw is not None else None

                if n is None or n <= 0:
                    if fallback_single_if_missing_count:
                        # One pseudo-turbine row for the whole block
                        rows.append(
                            {
                                "ID": f"NO_{pid}_blk_{b_i}_t_0",
                                "capacity": cap_per_turb_mw,  # may be None
                                "diameter": diameter,
                                "height": hub_height,
                                "manufacturer": manufacturer,
                                "lon": lon,
                                "lat": lat,
                                "type": "onshore",
                            }
                        )
                    continue

                for t_i in range(n):
                    rows.append(
                        {
                            "ID": f"NO_{pid}_blk_{b_i}_t_{t_i}",
                            "capacity": cap_per_turb_mw,  # MW per pseudo-turbine
                            "diameter": diameter,
                            "height": hub_height,
                            "manufacturer": manufacturer,
                            "lon": lon,
                            "lat": lat,
                            "type": "onshore",
                        }
                    )

        else:
            # Plant-level fallback (no blocks)
            rows.append(
                {
                    "ID": f"NO_{pid}_plant",
                    "capacity": plant_installed_mw,  # MW (plant total)
                    "diameter": None,
                    "height": None,
                    "manufacturer": None,
                    "lon": lon,
                    "lat": lat,
                    "type": "onshore",
                }
            )

    df = pd.DataFrame(rows).reindex(columns=SCHEMA)

    # Make sure numeric cols are numeric
    for c in ["capacity", "diameter", "height", "lon", "lat"]:
        df[c] = pd.to_numeric(df[c], errors="coerce")

    return df


def sanity_report(df: pd.DataFrame) -> None:
    print("Rows:", len(df))
    print("Missing lon/lat:", int(df["lon"].isna().sum()), int(df["lat"].isna().sum()))
    print("Missing capacity:", int(df["capacity"].isna().sum()))
    print("Missing diameter:", int(df["diameter"].isna().sum()))
    print("Missing height:", int(df["height"].isna().sum()))
    print("Manufacturer non-null:", int(df["manufacturer"].notna().sum()))
    print(df.head(5))


# -------------------------
# CLI / Run
# -------------------------
if __name__ == "__main__":
    out_csv = "NO_wind_metadata_pseudoturbines.csv"  # always write here

    data = fetch_nve_in_operation()
    df = nve_to_pseudoturbines(data, expand=True)

    df.to_csv(out_csv, index=False)
    print(f"Wrote {out_csv}")
    sanity_report(df)

Wrote NO_wind_metadata_pseudoturbines.csv
Rows: 1427
Missing lon/lat: 1427 1427
Missing capacity: 0
Missing diameter: 1427
Missing height: 1427
Manufacturer non-null: 0
                ID  capacity  diameter  height manufacturer  lon  lat     type
0  NO_20_blk_0_t_0       2.3       NaN     NaN         None  NaN  NaN  onshore
1  NO_20_blk_0_t_1       2.3       NaN     NaN         None  NaN  NaN  onshore
2  NO_20_blk_0_t_2       2.3       NaN     NaN         None  NaN  NaN  onshore
3  NO_20_blk_0_t_3       2.3       NaN     NaN         None  NaN  NaN  onshore
4  NO_20_blk_0_t_4       2.3       NaN     NaN         None  NaN  NaN  onshore


In [5]:
"""
NORWAY wind pseudo-turbines (onshore) from:
1) NVE WindPowerplant API (blocks: AntallTurbiner, TurbinStorrelse_kW, TurbinProdusent)
2) rebase-energy/nve-windpower-data CSV (coords + AvgHubHeight + AvgRotorDiameter)

Output columns:
["ID","capacity","diameter","height","manufacturer","lon","lat","type"]

No 'datasets' / pyarrow required.
"""

from __future__ import annotations

import io
import time
import requests
import pandas as pd

SCHEMA = ["ID", "capacity", "diameter", "height", "manufacturer", "lon", "lat", "type"]

# NVE (operational plants)
NVE_IN_OPERATION = "https://api.nve.no/web/WindPowerplant/GetWindPowerPlantsInOperation"

# Rebase-energy dataset (direct raw CSV on GitHub-backed HuggingFace repo)
# This file contains plant-level metadata including coordinates and averages.
# If the filename ever changes upstream, check the repo file list.
REBASE_META_CSV_URL = (
    "https://huggingface.co/datasets/rebase-energy/nve-windpower-data/resolve/main/"
    "nve-windpower-metadata.csv"
)


# -----------------------
# Fetchers
# -----------------------
def fetch_nve_in_operation(retries: int = 4, timeout: int = 60) -> list[dict]:
    last = None
    for i in range(retries):
        try:
            r = requests.get(NVE_IN_OPERATION, timeout=timeout)
            r.raise_for_status()
            data = r.json()
            if not isinstance(data, list):
                raise ValueError(f"Unexpected JSON type: {type(data)}")
            return data
        except Exception as e:
            last = e
            time.sleep(2 + i * 2)
    raise last


def fetch_rebase_meta_csv(url: str = REBASE_META_CSV_URL, timeout: int = 120) -> pd.DataFrame:
    r = requests.get(url, timeout=timeout)
    r.raise_for_status()
    return pd.read_csv(io.StringIO(r.text))


# -----------------------
# Transform
# -----------------------
def build_meta_enrichment(meta: pd.DataFrame) -> pd.DataFrame:
    """
    Standardise enrichment columns:
      - VindkraftAnleggId (int)  [join key]
      - lon, lat
      - avg_hub_height, avg_rotor_diameter
    """
    # Common columns in the rebase CSV:
    # WindPowerPlantId, lat, lon, AvgHubHeight, AvgRotorDiameter
    required = ["WindPowerPlantId", "lat", "lon", "AvgHubHeight", "AvgRotorDiameter"]
    missing = [c for c in required if c not in meta.columns]
    if missing:
        raise ValueError(
            "Rebase metadata CSV is missing expected columns: "
            f"{missing}\nAvailable columns: {list(meta.columns)}"
        )

    out = meta[required].copy()
    out = out.rename(
        columns={
            "WindPowerPlantId": "VindkraftAnleggId",
            "AvgHubHeight": "avg_hub_height",
            "AvgRotorDiameter": "avg_rotor_diameter",
        }
    )

    out["VindkraftAnleggId"] = pd.to_numeric(out["VindkraftAnleggId"], errors="coerce").astype("Int64")
    for c in ["lon", "lat", "avg_hub_height", "avg_rotor_diameter"]:
        out[c] = pd.to_numeric(out[c], errors="coerce")

    return out.dropna(subset=["VindkraftAnleggId"])


def nve_to_pseudoturbines(nve_data: list[dict], enrich: pd.DataFrame) -> pd.DataFrame:
    """
    Expand NVE turbine blocks into pseudo-turbines, enriched with coords + avg rotor/hub.

    capacity = per-turbine MW = TurbinStorrelse_kW / 1000
    diameter = avg_rotor_diameter (m)
    height   = avg_hub_height (m)
    lon/lat  = from enrich
    manufacturer = TurbinProdusent (block-level)
    type = onshore
    """
    enrich_idx = enrich.set_index("VindkraftAnleggId")

    rows = []
    for plant in nve_data:
        pid = plant.get("VindkraftAnleggId")
        if pid is None:
            continue

        try:
            pid_int = int(pid)
        except Exception:
            continue

        # enrichment lookup
        if pid_int in enrich_idx.index:
            lon = enrich_idx.loc[pid_int, "lon"]
            lat = enrich_idx.loc[pid_int, "lat"]
            diameter = enrich_idx.loc[pid_int, "avg_rotor_diameter"]
            height = enrich_idx.loc[pid_int, "avg_hub_height"]
        else:
            lon = lat = diameter = height = pd.NA

        blocks = plant.get("Turbiner") or []

        # If no blocks, fall back to one plant-level row
        if not blocks:
            rows.append(
                {
                    "ID": f"NO_{pid_int}_plant",
                    "capacity": plant.get("InstallertEffekt_MW"),
                    "diameter": diameter,
                    "height": height,
                    "manufacturer": pd.NA,
                    "lon": lon,
                    "lat": lat,
                    "type": "onshore",
                }
            )
            continue

        for b_i, blk in enumerate(blocks):
            n = blk.get("AntallTurbiner")
            size_kw = blk.get("TurbinStorrelse_kW")
            manu = blk.get("TurbinProdusent")

            cap_mw = pd.to_numeric(size_kw, errors="coerce") / 1000.0 if size_kw is not None else pd.NA

            # If count missing, emit one pseudo-turbine for that block
            if n is None or int(n) <= 0:
                rows.append(
                    {
                        "ID": f"NO_{pid_int}_blk_{b_i}_t_0",
                        "capacity": cap_mw,
                        "diameter": diameter,
                        "height": height,
                        "manufacturer": manu if manu not in (None, "") else pd.NA,
                        "lon": lon,
                        "lat": lat,
                        "type": "onshore",
                    }
                )
                continue

            for t_i in range(int(n)):
                rows.append(
                    {
                        "ID": f"NO_{pid_int}_blk_{b_i}_t_{t_i}",
                        "capacity": cap_mw,
                        "diameter": diameter,
                        "height": height,
                        "manufacturer": manu if manu not in (None, "") else pd.NA,
                        "lon": lon,
                        "lat": lat,
                        "type": "onshore",
                    }
                )

    df = pd.DataFrame(rows).reindex(columns=SCHEMA)
    for c in ["capacity", "diameter", "height", "lon", "lat"]:
        df[c] = pd.to_numeric(df[c], errors="coerce")
    return df


def sanity(df: pd.DataFrame):
    print("rows:", len(df))
    print("missing lon:", int(df["lon"].isna().sum()))
    print("missing lat:", int(df["lat"].isna().sum()))
    print("missing diameter:", int(df["diameter"].isna().sum()))
    print("missing height:", int(df["height"].isna().sum()))
    print(df.head(5))


# -----------------------
# Run (notebook-safe: no sys.argv)
# -----------------------
if __name__ == "__main__":
    out_csv = "NO_wind_metadata_pseudoturbines.csv"

    nve = fetch_nve_in_operation()
    meta_raw = fetch_rebase_meta_csv()
    enrich = build_meta_enrichment(meta_raw)

    df = nve_to_pseudoturbines(nve, enrich)
    df.to_csv(out_csv, index=False)

    print(f"Wrote {out_csv}")
    sanity(df)

Wrote NO_wind_metadata_pseudoturbines.csv
rows: 1427
missing lon: 10
missing lat: 10
missing diameter: 0
missing height: 0
                ID  capacity  diameter  height manufacturer        lon  \
0  NO_20_blk_0_t_0       2.3      71.0    64.0      Enercon  10.374618   
1  NO_20_blk_0_t_1       2.3      71.0    64.0      Enercon  10.374618   
2  NO_20_blk_0_t_2       2.3      71.0    64.0      Enercon  10.374618   
3  NO_20_blk_0_t_3       2.3      71.0    64.0      Enercon  10.374618   
4  NO_20_blk_0_t_4       2.3      71.0    64.0      Enercon  10.374618   

         lat     type  
0  64.224811  onshore  
1  64.224811  onshore  
2  64.224811  onshore  
3  64.224811  onshore  
4  64.224811  onshore  


In [ ]:


from __future__ import annotations

import pandas as pd
from pathlib import Path

OPSD_15MIN = "https://data.open-power-system-data.org/time_series/latest/time_series_15min_singleindex.csv"
OPSD_60MIN = "https://data.open-power-system-data.org/time_series/latest/time_series_60min_singleindex.csv"

SCHEMA_COLS = ["wind_onshore_MW", "wind_offshore_MW", "wind_total_MW"]

def load_opsd(url: str) -> pd.DataFrame:
    df = pd.read_csv(url, index_col=0, parse_dates=True, low_memory=False)
    df.index.name = "time"
    return df

def extract_wind(df: pd.DataFrame, country: str) -> pd.DataFrame:
    # Try both OPSD naming styles
    candidates = {
        "on": [
            f"{country}_wind_onshore_generation_actual",
            f"{country}_wind_onshore_generation",
        ],
        "off": [
            f"{country}_wind_offshore_generation_actual",
            f"{country}_wind_offshore_generation",
        ],
        "tot": [
            f"{country}_wind_generation_actual",
            f"{country}_wind_generation",
        ],
    }

    def first_present(cols):
        for c in cols:
            if c in df.columns:
                return c
        return None

    on_col = first_present(candidates["on"])
    off_col = first_present(candidates["off"])
    tot_col = first_present(candidates["tot"])

    # If nothing found, show hints
    if on_col is None and off_col is None and tot_col is None:
        hints = [c for c in df.columns if c.startswith(country + "_wind")]
        raise KeyError(f"No wind columns for {country}. Found: {hints[:40]}")

    out = pd.DataFrame(index=df.index)

    if on_col is not None:
        out["wind_onshore_MW"] = pd.to_numeric(df[on_col], errors="coerce")
    else:
        out["wind_onshore_MW"] = pd.NA

    if off_col is not None:
        out["wind_offshore_MW"] = pd.to_numeric(df[off_col], errors="coerce")
    else:
        out["wind_offshore_MW"] = pd.NA

    # If split exists, compute total; otherwise fall back to total column
    if on_col is not None or off_col is not None:
        out["wind_total_MW"] = out["wind_onshore_MW"].fillna(0) + out["wind_offshore_MW"].fillna(0)
    else:
        out["wind_total_MW"] = pd.to_numeric(df[tot_col], errors="coerce") if tot_col is not None else pd.NA

    return out[SCHEMA_COLS]

def to_hourly_mean(df: pd.DataFrame) -> pd.DataFrame:
    """
    OPSD generation is MW averaged over the reporting interval,
    so hourly aggregation should use mean.
    """
    return df.resample("1H").mean()

if __name__ == "__main__":
    out_be = Path("OPSD_BE_wind_hourly.csv")
    out_no = Path("OPSD_NO_wind_hourly.csv")

    # Belgium from 60-min
    df_60 = load_opsd(OPSD_60MIN)
    be = extract_wind(df_60, "BE")
    be.to_csv(out_be, index=True)

    no = extract_wind(df_60, "NO")
    no.to_csv(out_no, index=True)
    
    # 1) Confirm BE offshore exists
    print(be["wind_offshore_MW"].notna().mean())

    # 2) Confirm NO offshore is basically zero / missing (expected)
    print(no["wind_offshore_MW"].dropna().describe())

    print("Saved:")
    print(f" - {out_be}  {be.shape}")
    print(f" - {out_no}  {no.shape}")
    print("\nBE head:\n", be.head())
    print("\nNO head:\n", no.head())

0.9219261522588837
count       0
unique      0
top       NaN
freq      NaN
Name: wind_offshore_MW, dtype: object
Saved:
 - OPSD_BE_wind_hourly.csv  (50401, 3)
 - OPSD_NO_wind_hourly.csv  (50401, 3)

BE head:
                            wind_onshore_MW  wind_offshore_MW  wind_total_MW
time                                                                       
2014-12-31 23:00:00+00:00              NaN               NaN           0.00
2015-01-01 00:00:00+00:00              NaN               NaN           0.00
2015-01-01 01:00:00+00:00           216.15            518.66         734.81
2015-01-01 02:00:00+00:00           237.18            529.46         766.64
2015-01-01 03:00:00+00:00           326.19            406.94         733.13

NO head:
                            wind_onshore_MW wind_offshore_MW  wind_total_MW
time                                                                      
2014-12-31 23:00:00+00:00              NaN             <NA>           0.00
2015-01-01 00:00:00+00:

/var/folders/hb/8jmqlr8502jd4_jj817kmrq00000gn/T/ipykernel_42236/3616409726.py:62: FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  out["wind_total_MW"] = out["wind_onshore_MW"].fillna(0) + out["wind_offshore_MW"].fillna(0)


In [ ]:
from __future__ import annotations

import io
import time
import math
import random
import re
from typing import Iterable, Optional

import pandas as pd
import requests
import geopandas as gpd

# -----------------------------
# Shared config / schema
# -----------------------------
SCHEMA = ["ID", "capacity", "diameter", "height", "manufacturer", "lon", "lat", "type"]

# Belgium (Flanders) — Mercator OGC API Features
BE_MERCATOR_BASE = "https://www.mercator.vlaanderen.be/raadpleegdienstenmercatorpubliek/ogc/features/v1"
BE_MERCATOR_COLLECTION = "er:er_windturb_omv"

# Belgium offshore — RBINS WFS
BE_RBINS_WFS = "https://spatial.naturalsciences.be/geoserver/od_nature/ows"
BE_RBINS_LAYER = "od_nature:MUMM_windmill_locations_ETRS89"

# Norway — NVE operational plants (blocks)
NO_NVE_IN_OPERATION = "https://api.nve.no/web/WindPowerplant/GetWindPowerPlantsInOperation"

# Norway enrichment (coords + avg hub/rotor) — downloaded as CSV (no datasets/pyarrow required)
NO_REBASE_META_CSV_URL = (
    "https://huggingface.co/datasets/rebase-energy/nve-windpower-data/resolve/main/"
    "nve-windpower-metadata.csv"
)

# OSM Overpass mirrors
OVERPASS_MIRRORS = [
    "https://overpass.kumi.systems/api/interpreter",
    "https://overpass-api.de/api/interpreter",
    "https://overpass.openstreetmap.ru/api/interpreter",
    "https://overpass.nchc.org.tw/api/interpreter",
]

# -----------------------------
# Small helpers
# -----------------------------
def _to_num(x):
    if x is None or (isinstance(x, float) and math.isnan(x)):
        return pd.NA
    try:
        return float(x)
    except Exception:
        s = str(x).strip()
        m = re.search(r"[-+]?\d*\.?\d+", s.replace(",", "."))
        return float(m.group(0)) if m else pd.NA


def _ensure_lonlat_from_geometry(gdf: gpd.GeoDataFrame) -> gpd.GeoDataFrame:
    gdf = gdf.copy()
    if gdf.crs is None:
        gdf = gdf.set_crs(4326, allow_override=True)
    gdf = gdf.to_crs(4326)
    gdf["lon"] = gdf.geometry.x
    gdf["lat"] = gdf.geometry.y
    return gdf


def _dedup_by_distance(df: pd.DataFrame, *, km: float = 0.25, prefer_sources: Optional[list[str]] = None):
    """
    Simple greedy spatial de-dup on lon/lat within 'km' radius.
    If a 'source' column exists and prefer_sources given, higher priority kept.
    """
    if df.empty:
        return df

    R = 6371.0
    lat0 = math.radians(float(df["lat"].dropna().mean())) if df["lat"].notna().any() else 0.0

    def xy(lon, lat):
        return (math.radians(lon) * math.cos(lat0) * R, math.radians(lat) * R)

    # priority ordering
    if prefer_sources and "source" in df.columns:
        pr = {s: i for i, s in enumerate(prefer_sources)}
        df = df.copy()
        df["_prio"] = df["source"].map(lambda s: pr.get(s, 9999))
        df = df.sort_values(["_prio"]).drop(columns=["_prio"])
    else:
        df = df.copy()

    keep = []
    kept_xy = []
    km2 = km * km

    for _, row in df.iterrows():
        if pd.isna(row["lon"]) or pd.isna(row["lat"]):
            keep.append(True)
            continue
        x, y = xy(float(row["lon"]), float(row["lat"]))
        ok = True
        for (kx, ky) in kept_xy:
            dx = kx - x
            dy = ky - y
            if dx * dx + dy * dy <= km2:
                ok = False
                break
        keep.append(ok)
        if ok:
            kept_xy.append((x, y))

    return df.loc[keep].reset_index(drop=True)


def _overpass_post(query: str, *, timeout: int = 240, tries_per_mirror: int = 2):
    last_err = None
    for base in OVERPASS_MIRRORS:
        for attempt in range(tries_per_mirror):
            try:
                r = requests.post(base, data={"data": query}, timeout=timeout)
                if r.status_code == 429:
                    time.sleep(10 + attempt * 10)
                    continue
                r.raise_for_status()
                return r.json()
            except Exception as e:
                last_err = e
                time.sleep(5 + attempt * 5)
    raise last_err


# -----------------------------
# Belgium fetchers
# -----------------------------
def _fetch_be_flanders_mercator(*, timeout: int = 120) -> gpd.GeoDataFrame:
    """
    Mercator OGC API Features collection: er:er_windturb_omv
    """
    limit = 1000
    start = 0
    feats = []
    while True:
        url = f"{BE_MERCATOR_BASE}/collections/{BE_MERCATOR_COLLECTION}/items"
        params = {"f": "application/geo+json", "limit": limit, "startIndex": start}
        r = requests.get(url, params=params, timeout=timeout)
        r.raise_for_status()
        gj = r.json()
        batch = gj.get("features", [])
        feats.extend(batch)
        if len(batch) < limit:
            break
        start += limit

    gdf = gpd.GeoDataFrame.from_features(feats, crs="EPSG:4326")
    return gdf


def _normalise_be_flanders(gdf: gpd.GeoDataFrame) -> pd.DataFrame:
    gdf = _ensure_lonlat_from_geometry(gdf)
    # Known columns you showed:
    # fid, ashoogte (m), rotordiameter (m), vermogenmax (kW)
    out = pd.DataFrame({
        "ID": "BE_FLA_" + gdf["fid"].astype(str),
        "capacity": pd.to_numeric(gdf.get("vermogenmax"), errors="coerce") / 1000.0,  # kW -> MW
        "diameter": pd.to_numeric(gdf.get("rotordiameter"), errors="coerce"),
        "height": pd.to_numeric(gdf.get("ashoogte"), errors="coerce"),
        "manufacturer": pd.NA,
        "lon": pd.to_numeric(gdf["lon"], errors="coerce"),
        "lat": pd.to_numeric(gdf["lat"], errors="coerce"),
        "type": "onshore",
    }).reindex(columns=SCHEMA)
    return out


def _fetch_be_offshore_rbines(*, timeout: int = 120) -> gpd.GeoDataFrame:
    url = (
        f"{BE_RBINS_WFS}"
        f"?service=WFS&version=2.0.0&request=GetFeature"
        f"&typeName={BE_RBINS_LAYER}"
        f"&outputFormat=application/json"
        f"&srsName=EPSG:4326"
    )
    return gpd.read_file(url)


def _normalise_be_offshore(gdf: gpd.GeoDataFrame) -> pd.DataFrame:
    gdf = _ensure_lonlat_from_geometry(gdf)
    # RBINS layer often has sparse attributes; make IDs from index unless something obvious exists
    id_col = None
    for c in ["id", "ID", "fid", "FID", "objectid", "OBJECTID", "uuid", "name", "Naam", "gml_id"]:
        if c in gdf.columns:
            id_col = c
            break
    if id_col is None:
        ids = ["BE_OFF_" + str(i) for i in range(len(gdf))]
    else:
        ids = gdf[id_col].astype(str).tolist()

    out = pd.DataFrame({
        "ID": ids,
        "capacity": pd.NA,
        "diameter": pd.NA,
        "height": pd.NA,
        "manufacturer": pd.NA,
        "lon": pd.to_numeric(gdf["lon"], errors="coerce"),
        "lat": pd.to_numeric(gdf["lat"], errors="coerce"),
        "type": "offshore",
    }).reindex(columns=SCHEMA)
    return out


def _fetch_be_onshore_osm(*, timeout: int = 240) -> gpd.GeoDataFrame:
    template = """
    [out:json][timeout:180];
    area["ISO3166-1"="BE"][admin_level=2]->.be;
    (
      {ELEMENT}(area.be)["power"="generator"]["generator:source"="wind"];
    );
    out center tags;
    """
    elements = []
    for element in ["node", "way", "relation"]:
        data = _overpass_post(template.replace("{ELEMENT}", element), timeout=timeout)
        elements.extend(data.get("elements", []))

    records = []
    for el in elements:
        tags = el.get("tags", {}) or {}
        if el["type"] == "node":
            lon, lat = el.get("lon"), el.get("lat")
        else:
            center = el.get("center") or {}
            lon, lat = center.get("lon"), center.get("lat")
        if lon is None or lat is None:
            continue
        rec = {f"tag_{k}": v for k, v in tags.items()}
        rec.update({"osm_type": el["type"], "osm_id": el["id"], "lon": lon, "lat": lat})
        records.append(rec)

    df = pd.DataFrame(records)
    gdf = gpd.GeoDataFrame(df, geometry=gpd.points_from_xy(df["lon"], df["lat"]), crs="EPSG:4326")
    return gdf


def _normalise_be_osm(gdf: gpd.GeoDataFrame) -> pd.DataFrame:
    def parse_capacity_mw(x):
        if pd.isna(x):
            return pd.NA
        s = str(x).strip()
        m = re.search(r"([-+]?\d*\.?\d+)", s.replace(",", "."))
        if not m:
            return pd.NA
        val = float(m.group(1))
        if re.search(r"\bMW\b", s, re.I):
            return val
        if re.search(r"\bkW\b", s, re.I):
            return val / 1000.0
        if re.search(r"\bW\b", s) and not re.search(r"\bkW\b|\bMW\b", s, re.I):
            return val / 1e6
        # unknown unit; assume MW-like
        return val

    def parse_m(x):
        if pd.isna(x):
            return pd.NA
        s = str(x).strip()
        m = re.search(r"([-+]?\d*\.?\d+)", s.replace(",", "."))
        return float(m.group(1)) if m else pd.NA

    gdf = gdf.copy()
    out = pd.DataFrame(index=gdf.index)
    out["ID"] = "OSM_" + gdf["osm_type"].astype(str) + "_" + gdf["osm_id"].astype(str)

    cap = gdf.get("tag_generator:output:electricity", pd.Series([pd.NA] * len(gdf)))
    out["capacity"] = cap.map(parse_capacity_mw)

    dia = gdf.get("tag_rotor:diameter", gdf.get("tag_rotor_diameter", pd.Series([pd.NA] * len(gdf))))
    out["diameter"] = dia.map(parse_m)

    hub = gdf.get("tag_hub_height", gdf.get("tag_height", pd.Series([pd.NA] * len(gdf))))
    out["height"] = hub.map(parse_m)

    manu = gdf.get("tag_manufacturer", gdf.get("tag_brand", pd.Series([pd.NA] * len(gdf))))
    out["manufacturer"] = manu.astype(str)
    out.loc[out["manufacturer"].isin(["nan", "None", ""]), "manufacturer"] = pd.NA

    out["lon"] = pd.to_numeric(gdf["lon"], errors="coerce")
    out["lat"] = pd.to_numeric(gdf["lat"], errors="coerce")
    out["type"] = "onshore"

    out = out.reindex(columns=SCHEMA)
    return out


# -----------------------------
# Norway fetchers / normalisers
# -----------------------------
def _fetch_no_nve_operational(*, timeout: int = 60, retries: int = 4) -> list[dict]:
    last = None
    for i in range(retries):
        try:
            r = requests.get(NO_NVE_IN_OPERATION, timeout=timeout)
            r.raise_for_status()
            data = r.json()
            if not isinstance(data, list):
                raise ValueError(f"Unexpected JSON root type: {type(data)}")
            return data
        except Exception as e:
            last = e
            time.sleep(2 + i * 2)
    raise last


def _fetch_no_enrichment_csv(*, url: str = NO_REBASE_META_CSV_URL, timeout: int = 120) -> pd.DataFrame:
    r = requests.get(url, timeout=timeout)
    r.raise_for_status()
    return pd.read_csv(io.StringIO(r.text))


def _build_no_enrichment(meta: pd.DataFrame) -> pd.DataFrame:
    """
    Expect (common) columns:
      WindPowerPlantId, lat, lon, AvgHubHeight, AvgRotorDiameter
    If upstream changes, we keep best-effort.
    """
    # best-effort column picks
    def pick(cols: Iterable[str]) -> Optional[str]:
        for c in cols:
            if c in meta.columns:
                return c
        return None

    idc = pick(["WindPowerPlantId", "windpowerplantid", "VindkraftAnleggId", "VindkraftanleggId"])
    latc = pick(["lat", "Latitude", "latitude"])
    lonc = pick(["lon", "Longitude", "longitude"])
    hubc = pick(["AvgHubHeight", "avg_hub_height", "HubHeight", "GjsnittNavhoeyde", "GjsnittNavhoyde"])
    rotc = pick(["AvgRotorDiameter", "avg_rotor_diameter", "RotorDiameter", "GjsnittRotordiameter"])

    if idc is None:
        raise ValueError(f"Could not find plant-id column in Norway enrichment CSV. Columns: {list(meta.columns)}")

    out = pd.DataFrame({
        "VindkraftAnleggId": pd.to_numeric(meta[idc], errors="coerce").astype("Int64"),
        "lat": pd.to_numeric(meta[latc], errors="coerce") if latc else pd.NA,
        "lon": pd.to_numeric(meta[lonc], errors="coerce") if lonc else pd.NA,
        "avg_hub_height": pd.to_numeric(meta[hubc], errors="coerce") if hubc else pd.NA,
        "avg_rotor_diameter": pd.to_numeric(meta[rotc], errors="coerce") if rotc else pd.NA,
    })
    return out.dropna(subset=["VindkraftAnleggId"]).drop_duplicates(subset=["VindkraftAnleggId"])


def _normalise_no_pseudoturbines(
    nve_data: list[dict],
    enrich: pd.DataFrame,
    *,
    jitter_km: float = 0.0,
    seed: int = 0,
) -> pd.DataFrame:
    """
    Expand turbine blocks -> pseudo-turbines at plant centroid.
    capacity = per-turbine MW from TurbinStorrelse_kW / 1000 (if available)
    diameter/height/lon/lat from enrichment (plant-level averages & coords)
    manufacturer from block TurbinProdusent
    Optionally add small spatial jitter to pseudo-turbines (for plotting / spatial joins).
    """
    rng = random.Random(seed)
    idx = enrich.set_index("VindkraftAnleggId")

    rows = []
    for plant in nve_data:
        pid = plant.get("VindkraftAnleggId")
        if pid is None:
            continue
        try:
            pid = int(pid)
        except Exception:
            continue

        if pid in idx.index:
            lon = idx.loc[pid, "lon"]
            lat = idx.loc[pid, "lat"]
            diam = idx.loc[pid, "avg_rotor_diameter"]
            hub = idx.loc[pid, "avg_hub_height"]
        else:
            lon = lat = diam = hub = pd.NA

        blocks = plant.get("Turbiner") or []
        if not blocks:
            rows.append({
                "ID": f"NO_{pid}_plant",
                "capacity": _to_num(plant.get("InstallertEffekt_MW")),
                "diameter": _to_num(diam),
                "height": _to_num(hub),
                "manufacturer": pd.NA,
                "lon": _to_num(lon),
                "lat": _to_num(lat),
                "type": "onshore",
            })
            continue

        for b_i, blk in enumerate(blocks):
            n = blk.get("AntallTurbiner")
            try:
                n = int(n) if n is not None else 0
            except Exception:
                n = 0

            size_kw = blk.get("TurbinStorrelse_kW")
            cap_mw = pd.to_numeric(size_kw, errors="coerce") / 1000.0 if size_kw is not None else pd.NA
            manu = blk.get("TurbinProdusent")
            manu = manu if manu not in (None, "") else pd.NA

            if n <= 0:
                n = 1  # at least one pseudo-turbine for the block

            # jitter: uniform in circle radius jitter_km
            for t_i in range(n):
                jl = _to_num(lon)
                jt = _to_num(lat)
                if jitter_km and (jl is not pd.NA) and (jt is not pd.NA):
                    # crude jitter in degrees (OK for small distances)
                    # 1 deg lat ~ 111 km; lon scales by cos(lat)
                    r = jitter_km * math.sqrt(rng.random())
                    ang = 2 * math.pi * rng.random()
                    dlat = (r * math.sin(ang)) / 111.0
                    dlon = (r * math.cos(ang)) / (111.0 * max(0.2, math.cos(math.radians(float(jt)))))
                    jl = float(jl) + dlon
                    jt = float(jt) + dlat

                rows.append({
                    "ID": f"NO_{pid}_blk_{b_i}_t_{t_i}",
                    "capacity": cap_mw,
                    "diameter": _to_num(diam),
                    "height": _to_num(hub),
                    "manufacturer": manu,
                    "lon": jl,
                    "lat": jt,
                    "type": "onshore",
                })

    df = pd.DataFrame(rows).reindex(columns=SCHEMA)
    for c in ["capacity", "diameter", "height", "lon", "lat"]:
        df[c] = pd.to_numeric(df[c], errors="coerce")
    return df


# -----------------------------
# PUBLIC FUNCTIONS YOU ASKED FOR
# -----------------------------
def create_turbine_metadata_be(
    *,
    include_osm_wallonia_brussels: bool = True,
    dedup_km: float = 0.25,
    timeout: int = 120,
) -> pd.DataFrame:
    """
    Build Belgium turbine metadata (onshore + offshore) into one DataFrame with SCHEMA.
    Sources:
      - Flanders (onshore): Mercator OGC API (rich specs)
      - Offshore: RBINS WFS points
      - Wallonia+Brussels (onshore, best-effort): OSM Overpass (mirrors + retries)

    Returns: DataFrame with columns SCHEMA.
    """
    parts = []

    # Offshore (RBINS)
    be_off_gdf = _fetch_be_offshore_rbines(timeout=timeout)
    be_off = _normalise_be_offshore(be_off_gdf)
    be_off["source"] = "RBINS"
    parts.append(be_off)

    # Flanders onshore (Mercator)
    be_fla_gdf = _fetch_be_flanders_mercator(timeout=timeout)
    be_fla = _normalise_be_flanders(be_fla_gdf)
    be_fla["source"] = "MERCATOR"
    parts.append(be_fla)

    # Wallonia+Brussels onshore (OSM)
    if include_osm_wallonia_brussels:
        be_osm_gdf = _fetch_be_onshore_osm(timeout=max(timeout, 180))
        be_osm = _normalise_be_osm(be_osm_gdf)
        be_osm["source"] = "OSM"
        parts.append(be_osm)

    be = pd.concat(parts, ignore_index=True, sort=False)

    # Prefer authoritative sources if duplicates
    be = _dedup_by_distance(be, km=dedup_km, prefer_sources=["RBINS", "MERCATOR", "OSM"])

    # Ensure schema only
    be = be.reindex(columns=SCHEMA)
    return be


def create_turbine_metadata_no(
    *,
    jitter_km: float = 0.0,
    seed: int = 0,
    timeout: int = 120,
) -> pd.DataFrame:
    """
    Build Norway pseudo-turbine metadata (onshore) into SCHEMA.
    Sources:
      - NVE operational API: turbine blocks (count, size, manufacturer)
      - Enrichment CSV: plant lon/lat + avg hub height + avg rotor diameter (best-effort)

    Parameters:
      jitter_km: if >0, spatially jitter pseudo-turbines around plant centroid (useful for maps).
    """
    nve = _fetch_no_nve_operational(timeout=min(timeout, 120))
    meta_raw = _fetch_no_enrichment_csv(timeout=timeout)
    enrich = _build_no_enrichment(meta_raw)

    no = _normalise_no_pseudoturbines(nve, enrich, jitter_km=jitter_km, seed=seed)
    return no

def clean_and_impute_turbine_metadata(
    df: pd.DataFrame,
    *,
    country: str,
    drop_if_no_capacity: bool = True,
    verbose: bool = True,
) -> pd.DataFrame:
    """
    Cleans turbine metadata by:
    - imputing missing capacity / diameter / height
    - converting capacity from MW -> kW (FINAL OUTPUT)
    - optionally dropping rows still missing capacity

    Assumes INPUT capacity is in MW.
    Works for BE and NO outputs.
    """

    df = df.copy()

    # -------------------------------------------------
    # 0) Enforce numeric + clarify units (MW internally)
    # -------------------------------------------------
    df["capacity"] = pd.to_numeric(df["capacity"], errors="coerce")  # MW
    df["diameter"] = pd.to_numeric(df.get("diameter"), errors="coerce")
    df["height"] = pd.to_numeric(df.get("height"), errors="coerce")

    # -----------------------
    # 1) Capacity imputation (MW)
    # -----------------------
    if df["capacity"].isna().any():
        medians = (
            df.groupby("type")["capacity"]
            .median()
            .dropna()
        )

        def fill_capacity(row):
            if not pd.isna(row["capacity"]):
                return row["capacity"]
            return medians.get(row["type"], np.nan)

        df["capacity"] = df.apply(fill_capacity, axis=1)

    # -----------------------
    # 2) Diameter imputation
    # -----------------------
    # Empirical scaling: D ≈ 35 * sqrt(P_MW)
    mask_d = df["diameter"].isna() & df["capacity"].notna()
    df.loc[mask_d, "diameter"] = (
        35.0 * np.sqrt(df.loc[mask_d, "capacity"])
    )

    # Clamp to realistic bounds
    df["diameter"] = df["diameter"].clip(lower=40, upper=260)

    # -----------------------
    # 3) Hub height imputation
    # -----------------------
    mask_h = df["height"].isna() & df["diameter"].notna()

    df.loc[mask_h & (df["type"] == "onshore"), "height"] = (
        1.1 * df.loc[mask_h & (df["type"] == "onshore"), "diameter"]
    )

    df.loc[mask_h & (df["type"] == "offshore"), "height"] = (
        0.9 * df.loc[mask_h & (df["type"] == "offshore"), "diameter"]
    )

    # -----------------------
    # 4) Final drop (MW stage)
    # -----------------------
    if drop_if_no_capacity:
        before = len(df)
        df = df.dropna(subset=["capacity"])
        after = len(df)
        if verbose and before != after:
            print(f"[{country}] Dropped {before-after} rows with no capacity")

    # -----------------------
    # 5) Convert capacity MW → kW (FINAL OUTPUT)
    # -----------------------
    df["capacity"] = df["capacity"] * 1000.0  # MW → kW

    # -----------------------
    # 6) Sanity report
    # -----------------------
    if verbose:
        print(f"[{country}] Final turbine metadata (capacity in kW):")
        print(" rows:", len(df))
        print(" missing capacity:", int(df["capacity"].isna().sum()))
        print(" missing diameter:", int(df["diameter"].isna().sum()))
        print(" missing height:", int(df["height"].isna().sum()))
        print(
            " capacity range [MW]:",
            (df["capacity"] / 1000).min(),
            "–",
            (df["capacity"] / 1000).max(),
        )

    return df
# -----------------------------
# Example usage
# -----------------------------
if __name__ == "__main__":
    be = create_turbine_metadata_be(include_osm_wallonia_brussels=True, dedup_km=0.25)
    be = clean_and_impute_turbine_metadata(be, country="BE")
    be.to_csv("BE_turbine_metadata.csv", index=False)
    print("BE saved:", be.shape)

    no = create_turbine_metadata_no(jitter_km=0.0, seed=0)
    no = clean_and_impute_turbine_metadata(no, country="NO")
    no.to_csv("NO_turbine_metadata.csv", index=False)
    print("NO saved:", no.shape)

In [ ]:
be = create_turbine_metadata_be()
be = clean_and_impute_turbine_metadata(be, country="BE")

no = create_turbine_metadata_no()
no = clean_and_impute_turbine_metadata(no, country="NO")

NameError: name 'np' is not defined

In [ ]:
no.to_csv("no_md.csv", index=False)
be.to_csv("be_md.csv", index=False)